<a href="https://colab.research.google.com/github/zippy731/wiggle/blob/main/Wiggle_Standalone_5_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wiggle Standalone 5.0**
---
Generates semirandom animation keyframes for zoom/spin/translation for use with [Disco Diffusion v5 Colab notebook](https://colab.research.google.com/github/alembics/disco-diffusion/blob/main/Disco_Diffusion.ipynb) created by [@Somnai_dreams](https://twitter.com/Somnai_dreams) and [@gandamu](https://twitter.com/gandamu_ml)

**Concept:** Wiggle is based on 'episodes' of motion. Each episode is made of three distinct phases: attack (ramp up), decay (ramp down), and sustain (hold level steady). This is similar in concept to an ADSR envelope in a musical synthesizer.

Each parameter will ramp UP to a peak during attack phase, will ramp DOWN to a quiet level during decay phase, and will HOLD the quiet level during sustain phase.  At the end of the sustain phase, another episode will begin, ramping up to the next peak.

The parameters allow you to set the overall duration of each episode, the time split between phases, and the relative levels of the parameters in each phase.

Setting | Default / Typical | Description 
--- | --- | ---
***Time settings***||
wiggle_frames|1000|Total number of frames to model
episode_duration|48|average duration of each episode, in frames
wig_adsmix|(.2,.4,.4)|time split between attack,decay,sustain periods,should sum to 1.0
wig_time_var|0.20|allowable variance in time ranges. Must be < 1.0.  Set to 0 for precise control of frames 
***Zoom settings***||
wig_zoom_range|(.12,.18)|(2D) min/max peak zoom values. Negative values are zoom out.  In DD41, zoom values range around 1.0, so this zoom range is added to 1.0 later
wig_trz_range|(5,15)|(3D) min/max pixels of z translation (forward zoom) per frame. + is forward, 10 is good starting value.
wig_zoom_quiet_scale_factor|.20|multiplier factor to reduce motion values from peaks to lower 'quiet' period. 0-1, but can be > 1 for 'increase' during quiet period.
***Motion settings***||
wig_angle_range|(-3,3)|(2D) min/max rotation angle range - max degrees per frame
wig_trx_range|(-6,6)|min/max pixels of x translation per frame
wig_try_range|(-2,2)|min/max pixels of y translation per frame
wig_rotx_range|(-5,5)|min/max degrees of x rotation(up/down pitch) per frame
wig_roty_range|(-5,5)|min/max degrees of y rotation(left/right yaw) per frame
wig_rotz_range|(-3,3)|min/max degrees of z rotation(left/right roll) per frame
wig_quiet_scale_factor|.15|multiplier factor for quiet period of all non-zoom parameters

**Typical ADS/ settings:**
Different types of motion can be gained with combinations of the ADS timing and the two quiet scale settings

ADS Settings | WZQS | WQS |  Description 
--- | --- | --- | ---
***ADS tweaks***|||
(.2,.4,.4)| 0.20 | 0.15 |default 'purposeful' motion, gliding toward destination, pause at destination
(.1,.5,.4)| 0.20 | 0.15 |'sharp' motion, gliding toward destination, pause at destination
(.3,.3,.4)| 0.20 | 0.15 |meandering motion, gently gliding toward destination, pause at destination
***quiet factor tweaks***|||
(.2,.4,.4)| 0.85 | 0.15 |steady zooming, with periodic turns
(.2,.4,.4)| 3.0 | 0.15 |(use with lower base zoom ranges) - sharp turns with minimal zooming, then faster zooming with less turning

---
2/26/22
By [@zippy731](https://twitter.com/zippy731) 

In [9]:
#======= WIGGLE MODE
#@markdown ---
#@markdown ####**Wiggle:**
#@markdown Generates semirandom keyframes for zoom / spin / translation. Set params in code.
#@markdown

#.. can be embedded directly into DD5 notebook.  Copy this code into 
#animation settings tab, just before 'Coherency Settings'
#Then comment out standalone-only code and uncomment 'embedded-only' section.

#standalone-only:
import random
wiggle_frames = 1000#@param {type:"number"}
try:
    max_frames
except NameError:
    max_frames = wiggle_frames
wiggle_instead = True 
wiggle_show_params = True
#end standalone-only

#embedded-only:
#wiggle_instead = True #@param {type:"boolean"} 
#wiggle_show_params = True #@param {type:"boolean"} 
#end embedded-only code

if wiggle_instead:
    #episode time settings
    episode_duration = 48 # average duration of each episode, in frames
    episode_count = round((max_frames)/(episode_duration*.8),0)
    #attack/decay/sustain mix
    wig_adsmix = (.2,.4,.4) #should sum to 1.0
    wig_time_var = 0.2 # def 0.2 | allowable variance in time ranges. Must be < 1.0.  Set to 0 for precise control of frames.
    #lead_pause = random.randrange(11,13) # frames before first episode
    lead_pause = 12 # number of warmup frames before first episode. Recommended 6-12 to allow image to form before animation starts. 
    
    #calc time ranges    
    wig_attack_range=(round(episode_duration*wig_adsmix[0]*(1-wig_time_var),0),round(episode_duration*wig_adsmix[0]*(1+wig_time_var),0))
    wig_decay_range=(round(episode_duration*wig_adsmix[1]*(1-wig_time_var),0),round(episode_duration*wig_adsmix[1]*(1+wig_time_var),0))
    wig_sustain_range=(round(episode_duration*wig_adsmix[2]*(1-wig_time_var),0),round(episode_duration*wig_adsmix[2]*(1+wig_time_var),0))

    #zoom
    wig_zoom_range=(.12,.18) #(2D only) max zoom per frame.  Below, we'll add 100% to normalize 
    wig_trz_range=(5,15) #(3D only) max z translation. + is forward.
    wig_zqsf = .1 # wig_zoom_quiet_scale_factor//scale of zoom quiet periods, as function of above range

    #motion
    wig_angle_range=(-3,3) #(2D only) rotation angle range - max degrees per frame
    wig_trx_range=(-6,6) #x translation range - pixels per frame
    wig_try_range=(-2,2) #y translation range - pixels per frame
    wig_rotx_range=(-0,0) #(3D only) x rotation (pitch) range - degrees per frame
    wig_roty_range=(-0,0) #(3D only) y rotation (yaw) range - degrees per frame
    wig_rotz_range=(-0,0) #(3D only) z rotation (roll) range - degrees per frame

    wig_qsf = .15 # wig_quiet_scale_factor//scale of quiet periods, as function of above ranges
    #------------

    episodes = [(0,1.0,0,0,0,0,0,0,0)] #initialize episodes list
    #ep is: (frame,zoom,angle,trx,try,trz,rotx,roty,rotz)
    episode_starts = [0]
    episode_peaks = [0]
    i = 1
    skip_1 = 0
    wig_frame_count = round(lead_pause,0)
    while i < episode_count:
      #attack: quick ramp to motion
      if wig_time_var == 0:
        skip_1 = wig_attack_range[0]
      else:
        skip_1 = round(random.randrange(wig_attack_range[0],wig_attack_range[1]),0)
      wig_frame_count += int(skip_1)
      zoom_1 = 1+round(random.uniform(wig_zoom_range[0],wig_zoom_range[1]),3)
      trz_1 = round(random.uniform(wig_trz_range[0],wig_trz_range[1]),3)
      angle_1 = round(random.uniform(wig_angle_range[0],wig_angle_range[1]),3)
      trx_1 = round(random.uniform(wig_trx_range[0],wig_trx_range[1]),3)
      try_1 = round(random.uniform(wig_try_range[0],wig_try_range[1]),3)
      rotx_1 = round(random.uniform(wig_rotx_range[0],wig_rotx_range[1]),3) 
      roty_1 = round(random.uniform(wig_roty_range[0],wig_roty_range[1]),3) 
      rotz_1 = round(random.uniform(wig_rotz_range[0],wig_rotz_range[1]),3) 
      episodes.append((wig_frame_count,zoom_1,angle_1,trx_1,try_1,trz_1,rotx_1,roty_1,rotz_1))
      episode_starts.append((wig_frame_count))
      #decay: ramp down to element of interest
      if wig_time_var == 0:
        skip_1 = wig_decay_range[0]
      else:
        skip_1 = round(random.randrange(wig_decay_range[0],wig_decay_range[1]),0)
      wig_frame_count += int(skip_1)
      zoom_1 = 1+(round(wig_zqsf*random.uniform(wig_zoom_range[0],wig_zoom_range[1]),3))
      trz_1 = round(wig_zqsf*random.uniform(wig_trz_range[0],wig_trz_range[1]),3)
      angle_1 = round(wig_qsf*random.uniform(wig_angle_range[0],wig_angle_range[1]),3)
      trx_1 = round(wig_qsf*random.uniform(wig_trx_range[0],wig_trx_range[1]),3)
      try_1 = round(wig_qsf*random.uniform(wig_try_range[0],wig_try_range[1]),3)
      rotx_1 = round(wig_qsf*random.uniform(wig_rotx_range[0],wig_rotx_range[1]),3)
      roty_1 = round(wig_qsf*random.uniform(wig_roty_range[0],wig_roty_range[1]),3)
      rotz_1 = round(wig_qsf*random.uniform(wig_rotz_range[0],wig_rotz_range[1]),3)
      episodes.append((wig_frame_count,zoom_1,angle_1,trx_1,try_1,trz_1,rotx_1,roty_1,rotz_1))
      episode_peaks.append((wig_frame_count))
      #sustain: pause during element of interest
      if wig_time_var == 0:
        skip_1 = wig_sustain_range[0]
      else:
        skip_1 = round(random.randrange(wig_sustain_range[0],wig_sustain_range[1]),0)
      wig_frame_count += int(skip_1)
      zoom_1 = 1+(round(wig_zqsf*random.uniform(wig_zoom_range[0],wig_zoom_range[1]),3))
      trz_1 = round(wig_zqsf*random.uniform(wig_trz_range[0],wig_trz_range[1]),3)     
      angle_1 = round(wig_qsf*random.uniform(wig_angle_range[0],wig_angle_range[1]),3)
      trx_1 = round(wig_qsf*random.uniform(wig_trx_range[0],wig_trx_range[1]),3)
      try_1 = round(wig_qsf*random.uniform(wig_try_range[0],wig_try_range[1]),3)
      rotx_1 = round(wig_qsf*random.uniform(wig_rotx_range[0],wig_rotx_range[1]),3)
      roty_1 = round(wig_qsf*random.uniform(wig_roty_range[0],wig_roty_range[1]),3)
      rotz_1 = round(wig_qsf*random.uniform(wig_rotz_range[0],wig_rotz_range[1]),3)
      episodes.append((wig_frame_count,zoom_1,angle_1,trx_1,try_1,trz_1,rotx_1,roty_1,rotz_1))
      i+=1
    #trim off any episode > max_frames
    cleaned_episodes = [i for i in episodes if i[0] < max_frames]
    episodes = cleaned_episodes
    cleaned_episode_starts = [i for i in episode_starts if i < max_frames]
    episode_starts = cleaned_episode_starts
    cleaned_episode_peaks = [i for i in episode_peaks if i < max_frames]
    episode_peaks = cleaned_episode_peaks

    #build full schedule
    keyframe_frames = [item[0] for item in episodes]

    #Build keyframe strings 
    wig_zoom_string=''
    wig_angle_string=''
    wig_trx_string=''
    wig_try_string=''
    wig_trz_string=''
    wig_rotx_string=''
    wig_roty_string=''
    wig_rotz_string=''
    # iterate thru episodes, generate keyframe strings
    ### reformat as keyframe strings for testing
    i = 0
    while i < len(episodes):
      wig_zoom_string += str(int(episodes[i][0]))+':('+str(episodes[i][1])+'),'
      wig_angle_string += str(round(episodes[i][0],0))+':('+str(episodes[i][2])+'),'
      wig_trx_string += str(round(episodes[i][0],0))+':('+str(episodes[i][3])+'),'
      wig_try_string += str(round(episodes[i][0],0))+':('+str(episodes[i][4])+'),'
      wig_trz_string += str(round(episodes[i][0],0))+':('+str(episodes[i][5])+'),'
      wig_rotx_string += str(round(episodes[i][0],0))+':('+str(episodes[i][6])+'),'
      wig_roty_string += str(round(episodes[i][0],0))+':('+str(episodes[i][7])+'),'
      wig_rotz_string += str(round(episodes[i][0],0))+':('+str(episodes[i][8])+'),'
      i+=1    

    zoom = wig_zoom_string
    angle = wig_angle_string 
    translation_x = wig_trx_string
    translation_y =  wig_try_string
    translation_z =  wig_trz_string
    rotation_3d_x = wig_rotx_string
    rotation_3d_y = wig_roty_string
    rotation_3d_z = wig_rotz_string

    if wiggle_show_params:
      print('keyframe transitions:')
      print(keyframe_frames)
      print('episode_starts:')
      print(episode_starts)
      print('episode_peaks:')
      print(episode_peaks)
      print ('angle is')
      print(angle)    
      print ('zoom is')
      print(zoom)
      print ('translation_x is')
      print(translation_x)
      print ('translation_y is')
      print(translation_y)  
      print ('translation_z is')
      print(translation_z)  
      print ('rotation_3d_x is')
      print(rotation_3d_x)  
      print ('rotation_3d_y is')
      print(rotation_3d_y)  
      print ('rotation_3d_z is')
      print(rotation_3d_z)  
      print('end of wiggle params')

#============= END WIGGLE

keyframe transitions:
[0, 20, 39, 60, 69, 91, 108, 116, 131, 153, 164, 181, 199, 208, 226, 245, 254, 273, 291, 300, 318, 337, 345, 365, 381, 391, 409, 430, 438, 458, 473, 484, 500, 515, 525, 542, 557, 566, 584, 606, 616, 638, 656, 667, 688, 710, 719, 734, 749, 759, 776, 793, 801, 822, 844, 854, 870, 891, 899, 914, 935, 944, 960, 979, 988]
episode_starts:
[0, 20, 69, 116, 164, 208, 254, 300, 345, 391, 438, 484, 525, 566, 616, 667, 719, 759, 801, 854, 899, 944, 988]
episode_peaks:
[0, 39, 91, 131, 181, 226, 273, 318, 365, 409, 458, 500, 542, 584, 638, 688, 734, 776, 822, 870, 914, 960]
angle is
0:(0),20:(-0.232),39:(-0.176),60:(-0.107),69:(-2.158),91:(-0.196),108:(-0.131),116:(-1.881),131:(0.402),153:(-0.147),164:(2.895),181:(-0.17),199:(-0.213),208:(0.012),226:(-0.269),245:(0.308),254:(1.841),273:(-0.153),291:(-0.353),300:(-0.765),318:(-0.24),337:(0.1),345:(-1.31),365:(-0.062),381:(-0.326),391:(2.461),409:(-0.434),430:(-0.143),438:(2.864),458:(0.131),473:(-0.292),484:(-2.565),500:(0.38)